# Linear probing of sentiment classification in a transformer trained on causal_lm
In this notebook we'll try to find out if and how a transformer trained to do causal_lm process information about the sentiment of a sentence. We'll use the imdb dataset for this.

**GPU Requirements:** For running with GPT-2 you may be fine with just 8GB of GPU RAM. With about 24GB you should be able to run any 7B or 13B model. With 80GB (A100) GPU you may be able to run a 70B model.

In [1]:
from transformer_heads import load_headed
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    MistralForCausalLM,
    Trainer,
    BitsAndBytesConfig,
    TrainingArguments,
    GPT2Model,
    GPT2LMHeadModel,
)
from transformer_heads.util.helpers import DataCollatorWithPadding, get_model_params
from peft import LoraConfig
from transformer_heads.config import HeadConfig
from transformer_heads.util.model import print_trainable_parameters
from transformer_heads.util.evaluate import (
    evaluate_head_wise,
    get_top_n_preds,
    get_some_preds,
)
import torch
import pandas as pd

In [2]:
# GPT2 is the fastest and requires fewest memory. However, this works just the same with any Llama or Mistral model. Just change model_path to its huggingface path.
model_path = "gpt2"
train_epochs = 1
eval_epochs = 1
logging_steps = 100
full_finetuning = False
num_heads = 6

In [3]:
# Parameters
model_path = "gpt2"
train_epochs = 1
eval_epochs = 1
logging_steps = 100
full_finetune = False
num_heads = 6

In [4]:
model_params = get_model_params(model_path)
model_class = model_params["model_class"]
hidden_size = model_params["hidden_size"]
vocab_size = model_params["vocab_size"]
print(model_params)

{'vocab_size': 50257, 'n_positions': 1024, 'n_embd': 768, 'n_layer': 12, 'n_head': 12, 'n_inner': None, 'activation_function': 'gelu_new', 'resid_pdrop': 0.1, 'embd_pdrop': 0.1, 'attn_pdrop': 0.1, 'layer_norm_epsilon': 1e-05, 'initializer_range': 0.02, 'summary_type': 'cls_index', 'summary_use_proj': True, 'summary_activation': None, 'summary_first_dropout': 0.1, 'summary_proj_to_labels': True, 'scale_attn_weights': True, 'use_cache': True, 'scale_attn_by_inverse_layer_idx': False, 'reorder_and_upcast_attn': False, 'bos_token_id': 50256, 'eos_token_id': 50256, 'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': None, 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': True, 'chunk_size_feed_forward': 0, 'is_encoder_decoder': False, 'is_decoder': False, 'cross_attention_hidden_size': None, 'add_cross_attention': False, 'tie_encoder_decoder': False, 'max_length': 20, 'min_length': 0, '

We are doing text classification, so we have to set pred_for_sequence to True for this task. In the imdb dataset, we only have two labels, 0 for negative and 1 for positive. So we have to set num_outputs to 2.

In [5]:
head_configs = [
    HeadConfig(
        name=f"imdb_head_{(1+(i-1)*2)}",
        layer_hook=-(1 + (i - 1) * 2),
        in_size=hidden_size,
        output_activation="linear",
        pred_for_sequence=True,
        loss_fct="cross_entropy",
        num_outputs=2,
    )
    for i in range(1, num_heads + 2)
]

In [6]:
dd = load_dataset("imdb")

In [7]:
dd["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In the *tokenize_function*, we set the *label* entry in the dataset for each of our heads.

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    out = tokenizer(examples["text"], padding=False, truncation=True)
    for hc in head_configs:
        out[hc.name] = examples["label"]
    return out


for split in dd.keys():
    dd[split] = dd[split].filter(function=lambda example: len(example["text"]) > 10)
    dd[split] = dd[split].shuffle()
    dd[split] = dd[split].map(tokenize_function, batched=True)

dd.set_format(
    type="torch",
    columns=["input_ids", "attention_mask"] + [x.name for x in head_configs],
)
for split in dd.keys():
    dd[split] = dd[split].remove_columns(["text", "label"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

model = load_headed(
    model_class,
    model_path,
    head_configs=head_configs,
    quantization_config=None if full_finetuning else quantization_config,
    freeze_base_model=not full_finetuning,
    device_map={"": torch.cuda.current_device()},
)

Some weights of TransformerWithHeads were not initialized from the model checkpoint at gpt2 and are newly initialized: ['heads.imdb_head_1.lins.0.weight', 'heads.imdb_head_11.lins.0.weight', 'heads.imdb_head_13.lins.0.weight', 'heads.imdb_head_3.lins.0.weight', 'heads.imdb_head_5.lins.0.weight', 'heads.imdb_head_7.lins.0.weight', 'heads.imdb_head_9.lins.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print_trainable_parameters(model)

all params: 81983232 || trainable params: 10752 || trainable%: 0.01311487695435086
params by dtype: defaultdict(<class 'int'>, {torch.float32: 39515904, torch.uint8: 42467328})
trainable params by dtype: defaultdict(<class 'int'>, {torch.float32: 10752})


Our heads are linear layers with only two outputs. Thus we have a very low amount of trainable parameters.

In [11]:
ins, preds, ground_truths = get_some_preds(
    model, dd["test"], tokenizer, n=5, classification=True
)
print(
    pd.DataFrame(
        list(zip(ins, preds["imdb_head_5"], ground_truths["imdb_head_5"])),
        columns=["review", "label", "ground_truth"],
    )
)


Predicting:   0%|          | 0/5 [00:00<?, ?it/s]


Predicting:  20%|██        | 1/5 [00:00<00:01,  3.84it/s]


Predicting: 100%|██████████| 5/5 [00:00<00:00, 13.39it/s]

                                              review  label  ground_truth
0  This movie is a waste of time. Though it has a...      0             0
1  I saw this movie when it aired on Lifetime bac...      0             1
2  Leslie Nielsen hits rock bottom with this abso...      0             0
3  This BRASS EYE special PAEDO-GEDDON was swampe...      0             1
4  You can't help but marvel at Hitchcock's early...      0             1
5  i expected this movie to be absolutely god awf...      0             0


Untrained heads give fairly random outputs.

In [12]:
collator = DataCollatorWithPadding(
    feature_name_to_padding_value={
        "input_ids": tokenizer.pad_token_id,
        "attention_mask": 0,
    }
)

In [13]:
print(evaluate_head_wise(model, dd["test"], collator, epochs=eval_epochs))


Evaluating:   0%|          | 0/3125 [00:00<?, ?it/s]


Evaluating:   0%|          | 2/3125 [00:00<05:02, 10.32it/s]


Evaluating:   0%|          | 4/3125 [00:00<06:51,  7.59it/s]


Evaluating:   0%|          | 6/3125 [00:00<05:29,  9.46it/s]


Evaluating:   0%|          | 8/3125 [00:00<04:36, 11.28it/s]

...


Evaluating: 100%|█████████▉| 3123/3125 [05:20<00:00,  8.68it/s]


Evaluating: 100%|█████████▉| 3124/3125 [05:20<00:00,  8.68it/s]


Evaluating: 100%|██████████| 3125/3125 [05:21<00:00,  8.14it/s]


Evaluating: 100%|██████████| 3125/3125 [05:21<00:00,  9.73it/s]

(12.376125398864746, {'imdb_head_1': 2.2763312496745587, 'imdb_head_3': 2.3986372451511024, 'imdb_head_5': 2.028720821541548, 'imdb_head_7': 1.2156276172590257, 'imdb_head_9': 1.8285838321861625, 'imdb_head_11': 1.9336576251229645, 'imdb_head_13': 0.694567006111145})


In [14]:
args = TrainingArguments(
    output_dir="imdb_linear_probe",
    learning_rate=0.0002,
    num_train_epochs=train_epochs,  # To speed things up set to 0.1, set to 1 for better performance
    logging_steps=logging_steps,
    do_eval=False,
    remove_unused_columns=False,
)
trainer = Trainer(
    model,
    args=args,
    train_dataset=dd["train"],
    data_collator=collator,
)
trainer.train()

wandb: Currently logged in as: ykeller (chm-hci). Use `wandb login --relogin` to force relogin


wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Tracking run with wandb version 0.16.3


wandb: Run data is saved locally in /raven/u/ykeller/transformer_heads/wandb/run-20240324_002932-elgaoj7t
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run splendid-spaceship-212


...

Checkpoint destination directory imdb_linear_probe/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Checkpoint destination directory imdb_linear_probe/checkpoint-3000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=3125, training_loss=4.601652141113282, metrics={'train_runtime': 1123.3071, 'train_samples_per_second': 22.256, 'train_steps_per_second': 2.782, 'total_flos': 8513909276590080.0, 'train_loss': 4.601652141113282, 'epoch': 1.0})

In [15]:
print(evaluate_head_wise(model, dd["test"], collator, epochs=eval_epochs))


Evaluating:   0%|          | 0/3125 [00:00<?, ?it/s]

/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(

Evaluating:   0%|          | 2/3125 [00:00<05:12, 10.00it/s]


Evaluating:   0%|          | 4/3125 [00:00<07:10,  7.25it/s]


Evaluating:   0%|          | 6/3125 [00:00<05:42,  9.11it/s]


Evaluating:   0%|          | 8/3125 [00:00<04:46, 10.90it/s]

...


Evaluating: 100%|█████████▉| 3123/3125 [05:36<00:00,  8.16it/s]


Evaluating: 100%|█████████▉| 3124/3125 [05:36<00:00,  8.19it/s]


Evaluating: 100%|██████████| 3125/3125 [05:36<00:00,  7.67it/s]


Evaluating: 100%|██████████| 3125/3125 [05:36<00:00,  9.28it/s]

(4.350820428314209, {'imdb_head_1': 0.6196392296218872, 'imdb_head_3': 0.5805930749511719, 'imdb_head_5': 0.5834007847213745, 'imdb_head_7': 0.6084902699661255, 'imdb_head_9': 0.624310562505722, 'imdb_head_11': 0.6407481147956848, 'imdb_head_13': 0.6936383946609497})


In [16]:
ins, preds, ground_truths = get_some_preds(
    model, dd["test"], tokenizer, n=5, classification=True
)
print(
    pd.DataFrame(
        list(zip(ins, preds["imdb_head_5"], ground_truths["imdb_head_5"])),
        columns=["review", "label", "ground_truth"],
    )
)


Predicting:   0%|          | 0/5 [00:00<?, ?it/s]


Predicting:  80%|████████  | 4/5 [00:00<00:00, 37.13it/s]


Predicting: 100%|██████████| 5/5 [00:00<00:00, 31.48it/s]

                                              review  label  ground_truth
0  This movie is a waste of time. Though it has a...      0             0
1  I saw this movie when it aired on Lifetime bac...      1             1
2  Leslie Nielsen hits rock bottom with this abso...      0             0
3  This BRASS EYE special PAEDO-GEDDON was swampe...      0             1
4  You can't help but marvel at Hitchcock's early...      1             1
5  i expected this movie to be absolutely god awf...      0             0
